temperatura (e imagino que todos) datos cada 5 minutos

parametros: 
- gente que entra
- temperatura
- 

## Datos de ocupación

In [ ]:
import requests
import datetime
import time
import pandas as pd
import os

def obtener_datos_api(base_url, sensor_type_names=None, from_date=None, to_date=None, location_ids=None, tipo=None):
    """
    Extrae datos de una API en periodos de un mes y los guarda en un CSV consolidado.
    
    :param base_url: URL base de la API.
    :param sensor_type_names: Lista de nombres de sensores a consultar (opcional).
    :param from_date: Fecha de inicio en formato 'DD-MM-YYYY HH:MM' (opcional).
    :param to_date: Fecha de fin en formato 'DD-MM-YYYY HH:MM' (opcional).
    :param location_ids: Lista de IDs de ubicación (opcional).
    :param tipo: "aves" para conteo de aves, o cualquier otro valor para datos de series.
    :return: DataFrame consolidado con todos los datos.
    """
    
    def convertir_a_milisegundos(fecha_str):
        dt = datetime.datetime.strptime(fecha_str, "%d-%m-%Y %H:%M")
        return int(time.mktime(dt.timetuple()) * 1000)

    def convertir_a_fecha(epoch_ms):
        return datetime.datetime.fromtimestamp(epoch_ms / 1000).strftime("%d-%m-%Y %H:%M")

    if from_date is None:
        from_date = (datetime.datetime.now() - datetime.timedelta(days=30)).strftime("%d-%m-%Y %H:%M")
    if to_date is None:
        to_date = datetime.datetime.now().strftime("%d-%m-%Y %H:%M")

    from_epoch = convertir_a_milisegundos(from_date)
    to_epoch = convertir_a_milisegundos(to_date)

    datos_consolidados = {}
    registros = []
    periodo = 30 * 24 * 60 * 60 * 1000  # 1 mes en milisegundos

    while from_epoch < to_epoch:
        next_epoch = min(from_epoch + periodo, to_epoch)

        params = {
            "fromDate": from_epoch,
            "toDate": next_epoch
        }

        if sensor_type_names:
            params["sensorTypeNames"] = ",".join(sensor_type_names)
        if location_ids:
            params["locationIds"] = ",".join(map(str, location_ids))

        try:
            response = requests.get(base_url, params=params, verify=False)
            response.raise_for_status()
            data = response.json()
        except Exception as e:
            print(f"Error en la petición: {e}")
            from_epoch = next_epoch
            continue

        print(f"Procesando datos desde {convertir_a_fecha(from_epoch)} hasta {convertir_a_fecha(next_epoch)}")

        if tipo == "aves":
            fecha = convertir_a_fecha(from_epoch)
            if "count" in data and isinstance(data["count"], list):
                for ubicacion in data["count"]:
                    loc_id = ubicacion.get("location", "desconocido")
                    for especie, cantidad in ubicacion.get("count", []):
                        registros.append({
                            "timestamp": fecha,
                            "location": loc_id,
                            "species": especie,
                            "count": cantidad
                        })
            else:
                print("La respuesta no contiene la clave 'count' esperada para tipo='aves'.")

        else:
            if "series" in data:
                for variable, series_dict in data["series"].items():
                    for series_id, puntos in series_dict.items():
                        for timestamp, valor in puntos:
                            fecha = convertir_a_fecha(timestamp)
                            if fecha not in datos_consolidados:
                                datos_consolidados[fecha] = {}
                            datos_consolidados[fecha][variable] = valor
            else:
                print("La respuesta no contiene la clave 'series'.")

        from_epoch = next_epoch

    os.makedirs("data", exist_ok=True)

    if tipo == "aves":
        if not registros:
            print("No se obtuvieron datos de aves.")
            return None

        df_final = pd.DataFrame(registros)
        df_final = df_final.sort_values(["timestamp", "location", "species"])
        csv_filename = f"data/datos_aves_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_final.to_csv(csv_filename, index=False)

    else:
        if not datos_consolidados:
            print("No se obtuvieron datos.")
            return None

        df_final = pd.DataFrame.from_dict(datos_consolidados, orient="index")
        df_final.index.name = "timestamp"
        df_final = df_final.reset_index()
        df_final = df_final.sort_values("timestamp")
        csv_filename = f"data/datos_series_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_final.to_csv(csv_filename, index=False)

    print(f"Datos guardados en {csv_filename}")
    return df_final


In [14]:

url_api = "https://platon.grc.upv.es/sensingtools-api/api/user-hard-sensors/hup4zYsBoTGoLusGWiIj/data" 
sensores = ["count_in", "count_out"]
fecha_inicio = "10-09-2024 00:01"
fecha_fin = "11-12-2024 00:01"
ubicaciones = ['hup4zYsBoTGoLusGWiIj_entrada-outdoor_38.02497_-0.65828_0']

df_final = obtener_datos_api(url_api, sensores, fecha_inicio, fecha_fin, ubicaciones)


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Procesando datos desde 10-09-2024 00:01 hasta 10-10-2024 00:01


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Procesando datos desde 10-10-2024 00:01 hasta 08-11-2024 23:01


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Procesando datos desde 08-11-2024 23:01 hasta 08-12-2024 23:01


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Procesando datos desde 08-12-2024 23:01 hasta 11-12-2024 00:01
Datos guardados en data/datos_series_20250325_094824.csv


Ahora convertimos datos a integer  y creamos nueva variable `people_inside`  que representa el número de personas dentro del parque en el momento

In [17]:
df_final = pd.read_csv("data/datos_ocupacion.csv")
df_final["count_in"] = df_final["count_in"].astype('Int64')
df_final["count_out"] = df_final["count_out"].astype('Int64')
df_final["people_inside"] = df_final["count_in"] - df_final["count_out"]
df_final

,timestamp,count_in,count_out,people_inside
0,01-10-2024 00:01,0,0,0
1,01-10-2024 00:06,0,0,0
2,01-10-2024 00:11,0,0,0
3,01-10-2024 00:16,0,0,0
4,01-10-2024 00:21,0,0,0
...,...,...,...,...
39691,31-10-2024 23:47,0,1,-1
39692,31-10-2024 23:51,0,1,-1
39693,31-10-2024 23:52,0,1,-1
39694,31-10-2024 23:56,0,1,-1


## Datos de clima

In [20]:

url_api = "https://platon.grc.upv.es/sensingtools-api/api/user-hard-sensors/5rTM-4sB-iVpGSRJcSJV/data" 
sensores = ["temperatura", "velocidadviento", "direccionviento","lluvia", "intensidadlluvia", "radiacionsolar"]
fecha_inicio = "10-09-2024 00:01"
fecha_fin = "11-12-2024 00:01"
ubicaciones = ['5rTM-4sB-iVpGSRJcSJV_La Mata - Parque Natural_38.02519_-0.65845_0']

df_meteo = obtener_datos_api(url_api, sensores, fecha_inicio, fecha_fin, ubicaciones)



C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Procesando datos desde 10-09-2024 00:01 hasta 10-10-2024 00:01


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Procesando datos desde 10-10-2024 00:01 hasta 08-11-2024 23:01


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Procesando datos desde 08-11-2024 23:01 hasta 08-12-2024 23:01
Procesando datos desde 08-12-2024 23:01 hasta 11-12-2024 00:01
Datos guardados en data/datos_series_20250325_101658.csv


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [18]:
df_meteo

,timestamp,temperatura,velocidadviento,intensidadlluvia
6040,01-10-2024 00:00,20.8,0,0.0
6041,01-10-2024 00:05,20.8,0,0.0
6042,01-10-2024 00:10,20.7,0,0.0
6043,01-10-2024 00:15,20.5,0,0.0
6044,01-10-2024 00:20,20.4,0,0.0
...,...,...,...,...
14929,31-10-2024 23:35,18.2,0,0.0
14930,31-10-2024 23:40,18.1,0,0.0
14931,31-10-2024 23:45,18.1,0,0.0
14932,31-10-2024 23:50,17.9,0,0.0


 ## Datos de aves

In [16]:
url_api = "https://platon.grc.upv.es/sensingtools-api/api/user-hard-sensors/jBAFzo8B1ljoqTiFuu2B/data/count?/" 
sensores = []
fecha_inicio = "10-09-2024 00:01"
fecha_fin = "11-12-2024 00:01"
ubicaciones = []

df_aves = obtener_datos_api(url_api, sensores, fecha_inicio, fecha_fin, ubicaciones, "aves")

C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Procesando datos desde 10-09-2024 00:01 hasta 10-10-2024 00:01
Procesando datos desde 10-10-2024 00:01 hasta 08-11-2024 23:01
Procesando datos desde 08-11-2024 23:01 hasta 08-12-2024 23:01
Procesando datos desde 08-12-2024 23:01 hasta 11-12-2024 00:01
Datos guardados en data/datos_aves_20250325_095049.csv


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [19]:
df_aves

,timestamp,location,species,count
107,08-11-2024 23:01,jBAFzo8B1ljoqTiFuu2B_L3906629_38.02382_-0.68343_0,Alectoris rufa,1
108,08-11-2024 23:01,jBAFzo8B1ljoqTiFuu2B_L3906629_38.02382_-0.68343_0,Anthus pratensis,1
109,08-11-2024 23:01,jBAFzo8B1ljoqTiFuu2B_L3906629_38.02382_-0.68343_0,Ardea ibis,1
110,08-11-2024 23:01,jBAFzo8B1ljoqTiFuu2B_L3906629_38.02382_-0.68343_0,Carduelis carduelis,1
111,08-11-2024 23:01,jBAFzo8B1ljoqTiFuu2B_L3906629_38.02382_-0.68343_0,Columba palumbus,1
...,...,...,...,...
75,10-10-2024 00:01,jBAFzo8B1ljoqTiFuu2B_L6177434_38.02585_-0.66656_0,Erithacus rubecula,1
76,10-10-2024 00:01,jBAFzo8B1ljoqTiFuu2B_L6177434_38.02585_-0.66656_0,Myiopsitta monachus,1
69,10-10-2024 00:01,jBAFzo8B1ljoqTiFuu2B_L6177434_38.02585_-0.66656_0,Phalacrocorax carbo,2
77,10-10-2024 00:01,jBAFzo8B1ljoqTiFuu2B_L6177434_38.02585_-0.66656_0,Streptopelia decaocto,1


# METHOD

## Data Analitics:

- Data curation
- Data description
- Data Inference


## Data Visualization

- time series using timeGPT / Prohet / ...
LOOK FOR THE BEST TECHNIQUE FOR OUR DATA